In [9]:
from OMPython import OMCSessionZMQ
omc = OMCSessionZMQ()
from modelicares import SimRes
import pandas as pd
import numpy as np
import os
import shutil
import git

2020-04-07 12:55:22,515 - OMPython - INFO - OMC Server is up and running at file:////tmp/openmodelica.manuelnvro.port.d36353b955c94b56ada59e2785c513a4 pid=14093


In [10]:
#By default, the code runs in manuelnvro Dell using Dymola 2020. To change the computer change the following folders.
#OpenIPSL Location
OpenIPSL = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/OpenIPSL/"
#GitHub Location
GitHubOpenIPSL = "https://github.com/marcelofcastro/OpenIPSL.git"
OpenIPSLPackage = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/OpenIPSL/OpenIPSL/package.mo"
OpenModelica = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/"
#Working Directory
FTurbineGovernorsWorkingDir = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/WorkingDir/Fault/TurbineGovernors/"
#Load Variation Folder Locations
LoadVariationSource = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/Scripts/LoadVariation/AuxiliaryModels/Load_variation.mo"
LoadVariationDestinationPath = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/OpenIPSL/OpenIPSL/Electrical/Loads/PSSE/"
LoadVariationDestination = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/OpenIPSL/OpenIPSL/Electrical/Loads/PSSE/Load_variation.mo"
# Power Fault Folder Locations
PowerFaultSource = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/Scripts/LoadVariation/AuxiliaryModels/PwFault.mo"
PowerFaultDestinationPath = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/OpenIPSL/OpenIPSL/Electrical/Events/"
PowerFaultDestination = "/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerificationRoutines/OpenModelica/OpenIPSL/OpenIPSL/Electrical/Events/PwFault.mo"

In [11]:
print(omc.sendExpression("getVersion()"))
print("Open Modelica Exciters Simulation Start...")

OpenModelica 1.16.0~dev-263-g761b5de
Open Modelica Exciters Simulation Start...


In [4]:
#Deleting old OpenIPSL library version
shutil.rmtree(f""+OpenIPSL+"")
#Pulling latest OpenIPSL library version
print('Pulling latest OpenIPSL library version...\n')
git.Git(""+OpenModelica+"").clone(""+GitHubOpenIPSL+"")

Pulling latest OpenIPSL library version...



''

In [12]:
#Creation of matrix with names, paths and variables
tgovernors = { 'names' : ["BBGOV1","GAST", "GAST2A", "GGOV1", "HYGOV", "IEEEG1", "IEESGO", "TGOV1", "WEHGOV", 
                            "WESGOV", "WSHYDD", "WSHYGP"],
            'path' : ["OpenIPSL.Examples.Controls.PSSE.TG.BBGOV1","OpenIPSL.Examples.Controls.PSSE.TG.GAST",
                      "OpenIPSL.Examples.Controls.PSSE.TG.GAST2A", "OpenIPSL.Examples.Controls.PSSE.TG.GGOV1", "OpenIPSL.Examples.Controls.PSSE.TG.HYGOV", 
                      "OpenIPSL.Examples.Controls.PSSE.TG.IEEEG1", "OpenIPSL.Examples.Controls.PSSE.TG.IEESGO",
                      "OpenIPSL.Examples.Controls.PSSE.TG.TGOV1", "OpenIPSL.Examples.Controls.PSSE.TG.WEHGOV", 
                      "OpenIPSL.Examples.Controls.PSSE.TG.WESGOV", "OpenIPSL.Examples.Controls.PSSE.TG.WSHYDD", 
                      "OpenIPSL.Examples.Controls.PSSE.TG.WSHYGP", "OpenIPSL.Examples.Controls.PSSE.TG.WSHYGP"],
            'delta' : ['gENROU.delta', 'gENROE.delta', 'gENSAL.delta' ],
           'pelec' : ['gENROU.PELEC', 'gENROE.PELEC', 'gENSAL.PELEC'],
            'pmech' : ['gENROU.PMECH', 'gENROE.PMECH', 'gENSAL.PMECH'],
            'speed': ['gENROU.SPEED', 'gENROE.SPEED', 'gENSAL.SPEED'],
           'pmechgov' : ["bBGOV1.PMECH","gAST.PMECH", "gAST2A.PMECH", "gGOV1.PMECH", "hYGOV.PMECH", "iEEEG1.PMECH", 
                       "iEESGO.PMECH", "tGOV1.PMECH", "wEHGOV.PMECH", "wESGOV.PMECH", "wSHYDD.PMECH", 
                      "wSHYGP.PMECH"]}

In [33]:
#Delete old results
shutil.rmtree(''+FTurbineGovernorsWorkingDir+'')
#Create Exciters folder
os.makedirs(''+FTurbineGovernorsWorkingDir+'')
os.chdir(f""+FTurbineGovernorsWorkingDir+"")
for tgovernorNumber, tgovernorName in enumerate(tgovernors['names']):
    os.makedirs(f'{tgovernorName}')

In [34]:
#For loop that will iterate between machines, simulate, and create the .csv file
for tgovernorNumber, tgovernorName in enumerate(tgovernors['names']):
    print(f"Fault {tgovernorName} Simulation Start...")
    try:
        omc.sendExpression(f"cd(\"{FTurbineGovernorsWorkingDir}" + tgovernorName +"\")")
        omc.sendExpression(f"loadFile(\"{OpenIPSLPackage}\")")
        omc.sendExpression("instantiateModel(OpenIPSL)")
        omc.sendExpression(f"simulate(OpenIPSL.Examples.Controls.PSSE.TG.{tgovernorName}, stopTime=10.0,method=\"rungekutta\",numberOfIntervals=5000,tolerance=1e-06)")
        sim = SimRes(""+FTurbineGovernorsWorkingDir+f"{tgovernorName}/OpenIPSL.Examples.Controls.PSSE.TG.{tgovernorName}_res.mat")
        print(f"{tgovernorName} Simulation Finished...")
    except:
        print(f"{tgovernorName} simulation error or model not found...")
    try:
        #Selecting Variables
        print(".csv Writing Start...") 
        try:
            print('Verifying if it is a GENROU model...')
            #Selecting Variables
            print(tgovernors['delta'][0])
            variables = ['Time', tgovernors['delta'][0], tgovernors['pelec'][0], tgovernors['pmech'][0], tgovernors['speed'][0], tgovernors['pmechgov'][tgovernorNumber], 'GEN1.V', 'LOAD.V', 'GEN2.V', 'FAULT.V' ]
            df_variables = pd.DataFrame([], columns = variables)
            for var in variables:
                df_variables.drop(var, axis = 1, inplace = True)
                #Change from Radians to Degrees
                if var == tgovernors['delta'][0]:
                    df_variables[var] = np.array(sim[var].values()*(180/np.pi))    
                else:
                    #check if a variable does not change during the simulation and then and make a ones array and multiply by the value
                    try:
                        df_variables[var] = np.array(sim[var].values())
                    except:
                        first = np.array(sim[var].values())
                        df_variables[var] = first[0] * np.ones(df_variables['Time'].size)
            print(f"{tgovernorName} Variables OK...")
            #Changing current directory
            os.chdir(f""+FTurbineGovernorsWorkingDir+"")
            df_variables.to_csv(f'{tgovernorName}.csv', index = False)          
            print(f"{tgovernorName} Write OK...")
        except:
            print('Not a GENROU model...')
        try:    
            print('Verifying if it is a GENROE model...')
            #Selecting Variables
            print(tgovernors['delta'][1])
            variables = ['Time', tgovernors['delta'][1], tgovernors['pelec'][1], tgovernors['pmech'][1], tgovernors['speed'][1], tgovernors['pmechgov'][tgovernorNumber], 'GEN1.V', 'LOAD.V', 'GEN2.V', 'FAULT.V' ]
            df_variables = pd.DataFrame([], columns = variables)
            for var in variables:
                df_variables.drop(var, axis = 1, inplace = True)
                #Change from Radians to Degrees
                if var == tgovernors['delta'][0]:
                    df_variables[var] = np.array(sim[var].values()*(180/np.pi))    
                else:
                    #check if a variable does not change during the simulation and then and make a ones array and multiply by the value
                    try:
                        df_variables[var] = np.array(sim[var].values())
                    except:
                        first = np.array(sim[var].values())
                        df_variables[var] = first[0] * np.ones(df_variables['Time'].size)
            print(f"{tgovernorName} Variables OK...")
            #Changing current directory
            os.chdir(f""+FTurbineGovernorsWorkingDir+"")
            df_variables.to_csv(f'{tgovernorName}.csv', index = False)          
            print(f"{tgovernorName} Write OK...")
        except:
            print('Not a GENROE model...')
        try:
            print('Verifying if it is a GENSAL model...')
            #Selecting Variables
            print(tgovernors['delta'][2])
            variables = ['Time', tgovernors['delta'][2], tgovernors['pelec'][2], tgovernors['pmech'][2], tgovernors['speed'][2], tgovernors['pmechgov'][tgovernorNumber], 'GEN1.V', 'LOAD.V', 'GEN2.V', 'FAULT.V' ]
            df_variables = pd.DataFrame([], columns = variables)
            for var in variables:
                df_variables.drop(var, axis = 1, inplace = True)
                #Change from Radians to Degrees
                if var == tgovernors['delta'][0]:
                    df_variables[var] = np.array(sim[var].values()*(180/np.pi))    
                else:
                    #check if a variable does not change during the simulation and then and make a ones array and multiply by the value
                    try:
                        df_variables[var] = np.array(sim[var].values())
                    except:
                        first = np.array(sim[var].values())
                        df_variables[var] = first[0] * np.ones(df_variables['Time'].size)
            print(f"{tgovernorName} Variables OK...")
            #Changing current directory
            os.chdir(f""+FTurbineGovernorsWorkingDir+"")
            df_variables.to_csv(f'{tgovernorName}.csv', index = False)          
            print(f"{tgovernorName} Write OK...")
        except:
             print('Not a GENSAL model...')
    except:
        print(f"{tgovernorName} variable error...\n")
    shutil.rmtree(""+FTurbineGovernorsWorkingDir+f"{tgovernorName}/")
    print("Delete OK...\n")        
print('Fault Exciter Examples Open Modelica Simulation OK...')

Fault BBGOV1 Simulation Start...
BBGOV1 simulation error or model not found...
.csv Writing Start...
Verifying if it is a GENROU model...
gENROU.delta
Not a GENROU model...
Verifying if it is a GENROE model...
gENROE.delta
Not a GENROE model...
Verifying if it is a GENSAL model...
gENSAL.delta
Not a GENSAL model...
Delete OK...

Fault GAST Simulation Start...
GAST simulation error or model not found...
.csv Writing Start...
Verifying if it is a GENROU model...
gENROU.delta
Not a GENROU model...
Verifying if it is a GENROE model...
gENROE.delta
Not a GENROE model...
Verifying if it is a GENSAL model...
gENSAL.delta
Not a GENSAL model...
Delete OK...

Fault GAST2A Simulation Start...
GAST2A simulation error or model not found...
.csv Writing Start...
Verifying if it is a GENROU model...
gENROU.delta
Not a GENROU model...
Verifying if it is a GENROE model...
gENROE.delta
Not a GENROE model...
Verifying if it is a GENSAL model...
gENSAL.delta
Not a GENSAL model...
Delete OK...

Fault GGOV1

In [ ]:
try:
    print("Closing Open Modelica...")
    omc.sendExpression("quit()")
    print("OpenModelica Close OK...")
except:
    print("Open Modelica closing error...")